In [1]:
from memory_profiler import memory_usage
import time
import numpy as np

# ExaFMM-T
import exafmm.laplace as laplace

# PyExaFMM
from fmm import Fmm
from fmm.kernel import laplace_p2p_serial, laplace_gradient
from fmm.surface import scale_surface

# Adaptoctree
import adaptoctree.morton as morton
import adaptoctree.tree as tree

In [2]:
# Compile test experiment to cache numba functions
e = Fmm('test')
e.run()
laplace_p2p_serial(e.sources, e.targets, e.source_densities)
laplace_gradient(e.sources, e.targets, e.source_densities)

array([[  505.32437622,  1662.06575545, -1036.23303044],
       [ -319.91075759,   370.52167043,  1357.60678574],
       [  344.01185189,   913.15129831,   112.52339329],
       ...,
       [ -155.78901703,  -214.95415479,  -778.77791716],
       [ -168.37581363,  -681.57519089,  -444.27740462],
       [ -190.25537075,  -121.56253011,  -868.31197325]])

In [3]:
# ! rm test.hdf5 && fmm generate-test-data -c test && fmm compute-operators -c test
# ! rm 10_6_r.hdf5 && fmm generate-test-data -c 10_6_r && fmm compute-operators -c 10_6_r
# ! rm 10_6_s.hdf5 && fmm generate-test-data -c 10_6_s && fmm compute-operators -c 10_6_s
# ! rm 100_6_r.hdf5 && fmm generate-test-data -c 100_6_r && fmm compute-operators -c 100_6_r
# ! rm 100_6_s.hdf5 && fmm generate-test-data -c 100_6_s && fmm compute-operators -c 100_6_s
# ! rm full_6_r.hdf5 && fmm generate-test-data -c full_6_r && fmm compute-operators -c full_6_r
# ! rm full_6_s.hdf5 && fmm generate-test-data -c full_6_s && fmm compute-operators -c full_6_s


In [4]:
# Discretisation order of check surface
pvec = [6]
kvec = [10, 100, 'full']


# Load experimental data into PyExaFMM
pyfmmvec = []

for p in pvec:
    for k in kvec:
        pyfmmvec.append(Fmm(f'{k}_{p}_r'))
        pyfmmvec.append(Fmm(f'{k}_{p}_s'))

In [5]:
# Load experimental data into ExaFMM-T
exafmmvec = []
exafmmtreevec = []

for e in pyfmmvec:
    # create a list of source instances
    sources = laplace.init_sources(e.sources, e.source_densities)

    # create a list of target instances
    targets = laplace.init_targets(e.targets)
    
    # Expansion order
    p = e.config['order_equivalent']
    fmm = laplace.LaplaceFmm(p=p, ncrit=e.config['max_points'], filename=f'{p}.dat')
    exafmmvec.append(fmm)
    
    tree = laplace.setup(sources, targets, fmm)
    exafmmtreevec.append(tree)

# Runtimes

In [6]:
# Evaluate PyExaFMM experiments
nruns = 3
pytvec = [[] for _ in range(nruns)]
for i in range(nruns):    
    for e in pyfmmvec:
        s = time.time()
        e.run()
        pytvec[i].append(time.time()-s)
        
pytvec = np.array(pytvec)

In [7]:
# Evaluate ExaFMM-T experiments
nruns = 3
extvec = [[] for _ in range(nruns)]
for i in range(nruns):
    for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
        s = time.time()
        laplace.evaluate(tree, fmm)
        extvec[i].append(time.time()-s)
extvec = np.array(extvec)

In [8]:
pytvec.mean(axis=0)

array([17.74234605, 13.96312777, 22.83325942, 13.88112227, 54.43163498,
       16.17445699])

In [9]:
pytvec.std(axis=0)

array([0.53001843, 0.24038297, 0.15603686, 0.18613309, 3.00320396,
       0.13981452])

In [10]:
extvec.mean(axis=0)

array([1.23510718, 0.97440958, 0.82706753, 0.95615061, 1.49348116,
       1.43450491])

In [11]:
extvec.std(axis=0)

array([0.05400449, 0.03173536, 0.05482355, 0.0078165 , 0.08554058,
       0.31580335])

In [12]:
pytvec/extvec

array([[13.53295733, 15.08724659, 28.84370868, 14.34969448, 38.82627531,
        10.24859253],
       [14.41810458, 14.5444018 , 25.46048646, 14.75482242, 37.89902193,
        16.06457661],
       [15.21640667, 13.42477861, 28.83468361, 14.44923563, 32.91793506,
         9.45496792]])

# Memory

In [13]:
pyfmmvec[-1].nleaves

17017

In [14]:
pymemvec = []

for e in pyfmmvec:
    pymemvec.append(max(memory_usage(e.run)))

In [15]:
pymemvec

[8196.41796875,
 6209.359375,
 8196.578125,
 6209.359375,
 8196.578125,
 6267.8828125]

In [16]:
exmemvec = []

for tree, fmm in list(zip(exafmmtreevec, exafmmvec)):
    def f(): laplace.evaluate(tree, fmm)
    exmemvec.append(max(memory_usage(f)))

In [17]:
exmemvec

[5335.8203125,
 5252.3515625,
 4867.74609375,
 5251.7109375,
 5335.69140625,
 5252.32421875]

# Accuracy

In [ ]:
direct = laplace_p2p_serial(
    pyfmmvec[0].sources, 
    pyfmmvec[0].targets, 
    pyfmmvec[0].source_densities
)

In [14]:
pyfmmvec = [Fmm(f'C{C}E{C}') for C in Cvec]

for e in pyfmmvec:
    e.run()

In [17]:
pyfmmvec[-1].nleaves

1888